In [1]:
from transformers import AutoTokenizer, SwitchTransformersForConditionalGeneration,AutoModel, AutoConfig
from transformers.models import switch_transformers
from torch import nn
import torch
from torch.nn.functional import cosine_similarity
import types
import matplotlib.pyplot as plt
model = AutoModel.from_pretrained(
    "models/TinyLlama-1.1B-Chat-v1.0-x2-MoE"
)

/home/zhenghao2022/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
[k for k,_ in model.named_modules()]

['',
 'embed_tokens',
 'layers',
 'layers.0',
 'layers.0.self_attn',
 'layers.0.self_attn.q_proj',
 'layers.0.self_attn.k_proj',
 'layers.0.self_attn.v_proj',
 'layers.0.self_attn.o_proj',
 'layers.0.self_attn.rotary_emb',
 'layers.0.block_sparse_moe',
 'layers.0.block_sparse_moe.gate',
 'layers.0.block_sparse_moe.experts',
 'layers.0.block_sparse_moe.experts.0',
 'layers.0.block_sparse_moe.experts.0.w1',
 'layers.0.block_sparse_moe.experts.0.w2',
 'layers.0.block_sparse_moe.experts.0.w3',
 'layers.0.block_sparse_moe.experts.0.act_fn',
 'layers.0.block_sparse_moe.experts.1',
 'layers.0.block_sparse_moe.experts.1.w1',
 'layers.0.block_sparse_moe.experts.1.w2',
 'layers.0.block_sparse_moe.experts.1.w3',
 'layers.0.block_sparse_moe.experts.1.act_fn',
 'layers.0.input_layernorm',
 'layers.0.post_attention_layernorm',
 'layers.1',
 'layers.1.self_attn',
 'layers.1.self_attn.q_proj',
 'layers.1.self_attn.k_proj',
 'layers.1.self_attn.v_proj',
 'layers.1.self_attn.o_proj',
 'layers.1.self_att

In [1]:
# MixtralSparseMoeBlock->DecoderLayer->Model->PretrainModel->CasualLM
# 感觉学习微软的文章更现实

In [ ]:

experts = [
        "expert_0",
        "expert_1",
        "expert_2",
        "expert_3",
        "expert_4",
        "expert_5",
        "expert_6",
        "expert_7",
    ]
def layer_alter(model: nn.Module, alter_func, layer_type) -> None:
    for name, module in model.named_children():
        if len(list(module.children())) > 0:
            layer_alter(module, alter_func, layer_type)

        if isinstance(module, layer_type):
            alter_func(module)

In [ ]:
sparse_module = switch_transformers.SwitchTransformersSparseMLP
model_path = "/shared_home/arknet/hf_models/switch-base-8"
dtype = torch.bfloat16
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = SwitchTransformersForConditionalGeneration.from_pretrained(model_path, device_map="cuda:2", torch_dtype=dtype)
input_text = "A <extra_id_0> walks into a bar a orders a <extra_id_1> with <extra_id_2> pinch of <extra_id_3>."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

In [ ]:
def sparse_cos(module: nn.Module):
    num_experts = len(module.experts)
    main_expert = module.experts
    
    similarity_matrix = torch.zeros((8, 8))
    for i in range(8):
        for j in range(8):
            similarity_matrix[i, j] = cosine_similarity(
                module.experts[experts[i]].wi.weight.view(-1),
                module.experts[experts[j]].wi.weight.view(-1),
                dim=0,
            )
    plt.imshow(similarity_matrix.detach().numpy(), cmap='Blues', interpolation='nearest')

    # 在每个方块上显示具体的数字
    for i in range(8):
        for j in range(8):
            plt.text(j, i, format(similarity_matrix[i, j], '.2f'), ha='center', va='center', color='red')

    # 设置颜色映射范围
    plt.clim(0, 1)

    # 添加颜色条
    plt.colorbar()

    # 设置坐标轴标签和标题
    plt.xticks(range(8), experts, rotation=90)
    plt.yticks(range(8), experts)
    plt.title('Similarity Matrix')

    # 显示热力图
    plt.show()

In [ ]:
# 实现计划

In [ ]:
model

In [ ]:
# 调用 print_trainable_parameters（） 方法比较 PeftModel 的参数数与基础模型中的参数数！
# expert
# class SwitchTransformersDenseActDense(nn.Module):
#     def __init__(self, config: SwitchTransformersConfig):
#         self.wi = nn.Linear(config.d_model, config.d_ff, bias=False)
#         self.wo = nn.Linear(config.d_ff, config.d_model, bias=False)

# 这里的门控分数就是一个scale factor 和MEO的思路类似

In [ ]:
def sparse_avg(module: nn.Module):
    expert = module.experts['expert_0']
    
def sparse_first(module: nn.Module):
    expert = module.experts['expert_0']
    def first_forward(self, hidden_states):
        pass
def sparse_lora(module:nn.Module):
    num_experts = len(module.experts)
    main_expert = module.experts
    def lora_forward(self, hidden_states):
        router_mask, router_probs, router_logits = self.router(hidden_states)
        expert_index = torch.argmax(router_mask, dim=-1)
        # 这里的mask是一个tokens*num_experts的矩阵，表示每个token应该由哪个专家来处理

        next_states = hidden_states.clone()
        # 这个for函数，每次处理一个专家对应的所有token
        # 计划改成 expert + lora_up*lora_down
        # expert(hidden_states[token_indices]).to(next_states.dtype)
        for idx, expert in enumerate(self.experts.values()):
            token_indices = router_mask[:, :, idx].bool()
            next_states[token_indices] = expert(hidden_states[token_indices]).to(next_states.dtype)

        hidden_states = router_probs * next_states
        return hidden_states, (router_logits, expert_index)
    # d_model, d_ff = main_expert.wi.in_features, main_expert.wo.out_features

    module = [
        
    ]
    # module.forward = types.MethodType(switch_forward, module)
layer_alter(model, sparse_first, sparse_module)

In [ ]:
similarity_matrix = torch.zeros((8, 8))
plt.imshow(similarity_matrix, cmap="hot", interpolation="nearest")

In [ ]:
outputs = model.generate(input_ids)

In [ ]:
list(model.modules())[0]

In [ ]:
sparse_mlp = model.spa

In [ ]:
print(tokenizer.decode(outputs[0]))